# Data preprocessing
here we obtain the data from the json file and then process it by adding start and end tokens to it and then define the vocabulary, sentence list and also the tags (or the state) corresponding to each observation (or words). These will be helpful for us in determining the parameters of **Hidden Markov Model later**

In [13]:
import pandas as pd
import json
from collections import defaultdict,Counter

with open('input/train.json','r') as f:
    json_data = f.read()
val_data = json.load(open('input/dev.json'))

dataset = json.loads(json_data)
for item in dataset:
    item['sentence'].insert(0,'<s>')
    item['sentence'].append('</s>')
    item['labels'].insert(0,'<s>')
    item['labels'].append('</s>')
print(f'length of dataset is {len(dataset)}')


state_tags = defaultdict(set)
sentences = list()
state_seq = list()
vocab = set()
tags = set()
tag_count = defaultdict(int)
word_count = defaultdict(int)
tag_zipped_words = list()
from copy import deepcopy
for item in dataset:
    sentences.append(item['sentence'])
    state_seq.append(item['labels'])
    tag_zipped_words.append(deepcopy(list(zip(item['sentence'],item['labels']))))
    for word,count in list(Counter(item['sentence']).items()):
        word_count[word]+=count
    for tag,count in list(Counter(item['labels']).items()):
        tag_count[tag]+=count   
    vocab = vocab.union(set(item['sentence']))
    tags = tags.union(set(item['labels']))
    for i,word in enumerate(item['sentence']):
        state_tags[word].add(item['labels'][i])

length of dataset is 38218


# Determining Parameters of model
here we define the parameters of the model which are:
* $ \text{emission matrix B, where } B_i(o_t) = P(o_t|q_i)$
* $ \text{transition matrix A, where } A_{i-1,i} = P(q_{i}|q_{i-1})$
* $ \text{initial probabilities } \pi \text{ where } \pi_i = \text{initial probabilities of }i^{th}\text{label}$



In [18]:
A = defaultdict(lambda:defaultdict(float)) # transition matrix
B = defaultdict(lambda:defaultdict(float)) # emmision matrix
PI = defaultdict(float)

for item in state_seq:
    for i_1,i in zip(item,item[1:]):
        A[i_1][i]+=1

for item in A.keys():
    for key in A[item].keys():
        A[item][key] = A[item][key]/tag_count[item]
    
PI = A['<s>']
for item in tag_zipped_words:
    for obs,state in list(item):
        B[state][obs]+=1
for item in B.keys():
    for key in B[item].keys():
        B[item][key] /= tag_count[item]
    



# Viterbi algorithm

In [19]:
def viterbi(sentence):
    sentence = sentence.strip().split()
    memo = defaultdict(lambda: defaultdict(tuple))
    w = sentence[0]
    for tags in state_tags[w]:
        memo[0][tags] = (PI[tags],'<s>')
    for i in range(1,len(sentence)):
        w = sentence[i]
        tags = state_tags[w]
        for tag in tags:
            emission = B[tag][w]
            memo[i][tag] = (-1e9,'')
            for t,(prior,path) in memo[i-1].items():
                transition = A[t][tag]
                curr_prob = transition * emission * prior
                if curr_prob>memo[i][tag][0]:
                    memo[i][tag] = (curr_prob,f'{path},{tag}')
    n = len(sentence)
    res = ''
    check = -1e9
    for t,(prior,path) in memo[n-1].items():
        if prior>check:
            check = prior
            res = path
    return res
    

# demo test

In [20]:
viterbi("increase the pressure further more to get success")

'<s>,DT,NN,RB,JJR,TO,VB,NN'

# HMM model
Here we create an HMM model using and combine all the previously defined steps in the init method and running the virerbi in the forward method

In [26]:
class HMM:
    def __init__(self, dataset):
        # data processing
        self.state_tags = defaultdict(set)
        self.sentences = []
        self.state_seq = []
        self.vocab = set()
        self.tags = set()
        self.tag_count = defaultdict(int)
        self.word_count = defaultdict(int)
        self.tag_zipped_words = []
        from copy import deepcopy
        for item in dataset:
            self.sentences.append(item['sentence'])
            self.state_seq.append(item['labels'])
            self.tag_zipped_words.append(deepcopy(list(zip(item['sentence'], item['labels']))))
            for word, count in list(Counter(item['sentence']).items()):
                self.word_count[word] += count
            for tag, count in list(Counter(item['labels']).items()):
                self.tag_count[tag] += count
            self.vocab = self.vocab.union(set(item['sentence']))
            self.tags = self.tags.union(set(item['labels']))
            for i, word in enumerate(item['sentence']):
                self.state_tags[word].add(item['labels'][i])
        
        # parameters
        self.A = defaultdict(lambda: defaultdict(float))  # transition matrix
        self.B = defaultdict(lambda: defaultdict(float))  # emission matrix
        self.PI = defaultdict(float)

        for item in self.state_seq:
            for i_1, i in zip(item, item[1:]):
                self.A[i_1][i] += 1

        for item in self.A.keys():
            for key in self.A[item].keys():
                self.A[item][key] = self.A[item][key] / self.tag_count[item]

        self.PI = self.A['<s>']
        for item in self.tag_zipped_words:
            for obs, state in list(item):
                self.B[state][obs] += 1
        for item in self.B.keys():
            for key in self.B[item].keys():
                self.B[item][key] /= self.tag_count[item]

    def forward(self, sentence):
        sentence = sentence.strip().split()
        memo = defaultdict(lambda: defaultdict(tuple))
        w = sentence[0]
        for tags in self.state_tags[w]:
            memo[0][tags] = (self.PI[tags],'<s>')
        for i in range(1,len(sentence)):
            w = sentence[i]
            tags = self.state_tags[w]
            for tag in tags:
                emission = self.B[tag][w]
                memo[i][tag] = (-1e9,'')
                for t,(prior,path) in memo[i-1].items():
                    transition = self.A[t][tag]
                    curr_prob = transition * emission * prior
                    if curr_prob>memo[i][tag][0]:
                        memo[i][tag] = (curr_prob,f'{path},{tag}')
        n = len(sentence)
        res = ''
        check = -1e9
        for t,(prior,path) in memo[n-1].items():
            if prior>check:
                check = prior
                res = path
        return res

    def __getstate__(self):
        """Prepare for pickling."""
        state = self.__dict__.copy()
        # Convert defaultdicts to dicts for pickling
        state['state_tags'] = dict(state['state_tags'])
        state['A'] = {k: dict(v) for k, v in state['A'].items()}
        state['B'] = {k: dict(v) for k, v in state['B'].items()}
        state['PI'] = dict(state['PI'])
        return state

    def __setstate__(self, state):
        """Reconstruct from pickle."""
        self.__dict__.update(state)
        # Convert dicts back to defaultdicts if needed
        self.state_tags = defaultdict(set, self.state_tags)
        self.A = defaultdict(lambda: defaultdict(float), {k: defaultdict(float, v) for k, v in self.A.items()})
        self.B = defaultdict(lambda: defaultdict(float), {k: defaultdict(float, v) for k, v in self.B.items()})
        self.PI = defaultdict(float, self.PI)


# Training and testing the HMM 
instantiating the HMM object by passing a training dataset will build the parameter matrices automatically 

In [ ]:
model = HMM(dataset)

In [24]:
model.forward('I will work hard to do well ').split(',')[1:]

['MD', 'VB', 'JJ', 'TO', 'VB', 'RB']

# Saving The Model

In [24]:
import pickle
with open('/kaggle/working/hmm_model.pkl','wb') as f:
    pickle.dump(model,f)

# Checking Metrics

In [45]:
from sklearn.metrics import classification_report, f1_score
import pickle
from collections import defaultdict
predicted_tags = list()
true_tags = list()
count = 0
total = 0
with open('hmm_model.pkl', 'rb') as f:
    model = pickle.load(f)
for element in val_data:
    sentence = ' '.join(element['sentence'])
    pred_tag = model.forward(sentence).split(',')[1:]
    if len(element["labels"])==len(pred_tag):
        predicted_tags.append(pred_tag)
        true_tags.append(element['labels'])
# Flatten the lists of lists into a single list for evaluation
true_tags_flat = [tag for sent in true_tags for tag in sent]
predicted_tags_flat = [tag for sent in predicted_tags for tag in sent]

# Calculate the F1 score
f1 = f1_score(true_tags_flat, predicted_tags_flat, average='weighted')

# Print the classification report for a detailed breakdown
report = classification_report(true_tags_flat, predicted_tags_flat)

print(f'F1 Score: {f1}')
print('Classification Report:')
print(report)

c:\Users\ipsit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ipsit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ipsit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, 

F1 Score: 0.523539597302735
Classification Report:
              precision    recall  f1-score   support

                   0.00      0.00      0.00         0
           $       0.56      0.56      0.56       156
          ''       0.92      0.91      0.91       168
           ,       0.00      0.00      0.00      1021
       -LRB-       0.33      0.21      0.26        19
       -RRB-       0.74      0.74      0.74        19
           .       1.00      1.00      1.00       945
           :       0.64      0.62      0.63        85
          CC       0.69      0.60      0.64       519
          CD       0.61      0.61      0.61       767
          DT       0.55      0.47      0.51      2003
          EX       0.64      0.52      0.57        27
          FW       0.00      0.00      0.00         0
          IN       0.52      0.46      0.49      2377
          JJ       0.46      0.50      0.48      1386
         JJR       0.57      0.55      0.56       125
         JJS       0.44      0

c:\Users\ipsit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ipsit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
